# Crawling

In [ ]:
# !pip install beautifulsoup4

In [22]:
# 셀레니움
from selenium import webdriver 
# 셀레니움 By 클래스 가져오기
from selenium.webdriver.common.by import By
# BeautifulSoup
from bs4 import BeautifulSoup

In [10]:
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm

# 순서
1. 주소창에 '종로구 빵' 입력
2. 검색을 위해 엔터키 누르도록 명령
3. 빵 집 누르고 
4. 해당 빵집 정보 크롤링

In [75]:
# 크롬 웹 드라이버 초기화 및 카카오 맵 페이지로 이동
driver = webdriver.Chrome() 
url = 'https://m.map.kakao.com/'
driver.get(url)

# 검색창에 '종로구 빵'을 입력

# 검색창 element와 검색창 id 가져오기
search_box = driver.find_element(By.ID,'innerQuery')
# 키워드 입력
search_box.send_keys("종로구 빵")

# Enter 키 누르기
search_box.send_keys(Keys.RETURN)

# 페이지가 로드될 때까지 잠시 대기
time.sleep(3)

# 베너 닫기
driver.find_element(By.XPATH, f'//*[@id="daumWrap"]/div[1]/div/a[2]/span').click()

## 더이상 더보기 버튼 누를 수 없을 때 까지 누르기

In [76]:
# 더보기 버튼을 누를 수 있는지 여부 확인하는 함수
def can_click_load_more_button():
    try:
        load_more_button = driver.find_element(By.XPATH, '//*[@id="daumContent"]/div[4]/div[2]/div[3]/a')  # 더보기 버튼의 XPATH
        return load_more_button.is_enabled() and load_more_button.is_displayed()
    except:
        return False

# 더보기 버튼 클릭 함수
def click_load_more_button():
    load_more_button = driver.find_element(By.XPATH, '//*[@id="daumContent"]/div[4]/div[2]/div[3]/a')  # 더보기 버튼의 XPATH
    load_more_button.click()
    time.sleep(3)  # 버튼을 클릭한 후에 충분한 시간 대기 (페이지가 로딩될 때까지)

# 더보기 버튼을 누를 수 없을 때까지 반복해서 클릭
while can_click_load_more_button():
    click_load_more_button()
    
# 페이지에 표시된 정보의 갯수 확인
all_results = driver.find_elements(By.XPATH, '//*[@id="placeList"]/li')
print("페이지에 표시된 정보의 갯수:", len(all_results))

페이지에 표시된 정보의 갯수: 76


In [74]:
type(all_results)

list

9번째 요소까지 a / 10번째 요소부터 a[1]
2, 3번째 사이에 배너가 있어서 숫자가 인덱스가 연속적이지 않음

- 가게 이름 XPATH
//*[@id="placeList"]/li[1]/a[1]/span[2]/span[1]/strong
//*[@id="placeList"]/li[2]/a[1]/span[2]/span[1]/strong
//*[@id="placeList"]/li[4]/a[1]/span[2]/span[1]/strong
//*[@id="placeList"]/li[10]/a[1]/span[2]/span[1]/strong
//*[@id="placeList"]/li[16]/a/span[2]/span[1]/strong
//*[@id="placeList"]/li[17]/a[1]/span[2]/span[1]/strong 더보기 누른 후 첫번쨰
//*[@id="placeList"]/li[61]/a/span[2]/span[1]/strong 더보기 누른 후 마지막
//*[@id="placeList"]/li[72]/a[1]/span[2]/span[1]/strong
//*[@id="placeList"]/li[76]/a[1]/span[2]/span[1]/strong
//*[@id="placeList"]/li[63]/a[1]/span[2]/span[1]/strong
//*[@id="placeList"]/li[23]/a[1]/span[2]/span[1]/strong
//*[@id="placeList"]/li[50]/a[1]/span[2]/span[1]/strong

- 리뷰 점수
//*[@id="placeList"]/li[16]/a/span[2]/span[2]/span[1]/em
//*[@id="placeList"]/li[10]/a[1]/span[2]/span[2]/span[1]/em
//*[@id="placeList"]/li[4]/a[1]/span[2]/span[2]/span[1]/em
//*[@id="placeList"]/li[2]/a[1]/span[2]/span[2]/span[1]/em
//*[@id="placeList"]/li[1]/a[1]/span[2]/span[2]/span[1]/em
//*[@id="placeList"]/li[50]/a[1]/span[2]/span[2]/span[1]/em

- 리뷰 갯수
//*[@id="placeList"]/li[1]/a[1]/span[2]/span[2]/span[2]
//*[@id="placeList"]/li[2]/a[1]/span[2]/span[2]/span[2]
//*[@id="placeList"]/li[4]/a[1]/span[2]/span[2]/span[2]
//*[@id="placeList"]/li[16]/a/span[2]/span[2]/span[2]
//*[@id="placeList"]/li[16]/a/span[2]/span[2]/span[2]
//*[@id="placeList"]/li[11]/a/span[2]/span[2]/span[2]
//*[@id="placeList"]/li[10]/a[1]/span[2]/span[2]/span[2]

- 주소
//*[@id="placeList"]/li[1]/a[1]/span[2]/span[3]
//*[@id="placeList"]/li[4]/a[1]/span[2]/span[3]
//*[@id="placeList"]/li[10]/a[1]/span[2]/span[3]
//*[@id="placeList"]/li[11]/a/span[2]/span[3]
//*[@id="placeList"]/li[16]/a/span[2]/span[3]

In [86]:
# NoSuchElementException import 추가
from selenium.common.exceptions import NoSuchElementException  

In [87]:
# 크롤링
# 가게 이름
def store_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[1]/strong'
    try:
        return driver.find_element(By.XPATH, xpath).text
    # 베너가 있는 경우 다른 XPATH로 시도
    except NoSuchElementException:
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span[2]/span[1]/strong'   
        return driver.find_element(By.XPATH, xpath).text
    
# 리뷰 점수  
def rv_score_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[2]/span[1]/em'
    try:
        return driver.find_element(By.XPATH, xpath).text
    except NoSuchElementException:
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span[2]/span[2]/span[1]/em'
        try:
            return driver.find_element(By.XPATH, xpath).text
        # 예외 처리: 요소를 찾지 못할 경우 None 반환
        except NoSuchElementException:
            return None 


# 리뷰 갯수 
def rv_count_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[2]/span[2]'
    try:
        return driver.find_element(By.XPATH, xpath).text
    except NoSuchElementException:
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span[2]/span[2]/span[2]'
        try:
            return driver.find_element(By.XPATH, xpath).text
        except NoSuchElementException:
            return None 

# 주소 
def add_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[3]'
    try:
        return driver.find_element(By.XPATH, xpath).text
    except NoSuchElementException:
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span[2]/span[3]'
        return driver.find_element(By.XPATH, xpath).text

store = []
review_score = []
review_count = []
address = []

# # XPATH 범위 지정 
# range_list = [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

for i in range(1, len(all_results) + 1): 
    if i == 3:
        continue
    
    time.sleep(3)
    
    store_name = store_def(i)
    store.append(store_name)
    
    review_sc = rv_score_def(i)
    review_score.append(review_sc)
    
    review_ct = rv_count_def(i)
    review_count.append(review_ct)
    
    add = add_def(i)
    address.append(add)
    

In [90]:
print(len(store))
print(len(review_score))
print(len(review_count))
print(len(address))

75
75
75
75


In [95]:
import re

In [98]:
# 숫자로 변환하는 함수
def extract_number(item):
    if item is None:
        return None
    match = re.search(r'\d+', item)
    return int(match.group()) if match else None

# 각 항목을 숫자로 변환하여 새로운 리스트 생성
review_count = [extract_number(item) for item in review_count]

In [104]:
review_count[45:50]

[13, 9, None, 7, 18]

In [105]:
import pandas as pd

In [ ]:
# DF에 담기
kakao = pd.DataFrame()

kakao['Store'] = store
kakao['Adddress'] = address
kakao['Review_score'] = review_score
kakao['Review_counts'] = review_count

In [117]:
kakao

,Store,Adddress,Review_score,Review_counts
0,스코프 서촌점,서울 종로구 필운대로5가길 31 (누하동),3.9,223.0
1,부암동스코프,서울 종로구 창의문로 149 1층 (부암동),4.0,204.0
2,효자베이커리,서울 종로구 필운대로 54 (통인동),3.8,139.0
3,아베베베이커리 서울점,서울 종로구 청계천로 201 1~3층 (예지동),4.0,109.0
4,런던베이글뮤지엄 안국점,서울 종로구 북촌로4길 20 연화빌딩 1층 (계동),3.0,605.0
...,...,...,...,...
70,아티제 세종문화회관점,서울 종로구 세종대로 175 세종문화회관 1층 (세종로),3.4,7.0
71,파리바게뜨 동묘역점,서울 종로구 지봉로 43 (창신동),3.0,2.0
72,북촌유기농명인단팥빵 광화문점,서울 종로구 새문안로9길 21 1층 (당주동),4.3,6.0
73,모자이크베이커리,서울 종로구 율곡로 221-12 1층 (이화동),4.6,14.0


In [115]:
kakao[kakao['Review_score'].isna()]

,Store,Adddress,Review_score,Review_counts
47,파리바게뜨 경복궁역점,서울 종로구 자하문로 5 (체부동),NaN,NaN
69,뚜레쥬르 광화문경희궁의아침점,"서울 종로구 사직로8길 34 경희궁의아침3단지상가 1층 103,104,129,130...",NaN,NaN


In [111]:
# Review_score 시리즈의 데이터 유형을 float64로 변경
kakao['Review_score'] = kakao['Review_score'].astype(float)

In [112]:
kakao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Store          75 non-null     object 
 1   Adddress       75 non-null     object 
 2   Review_score   73 non-null     float64
 3   Review_counts  73 non-null     float64
dtypes: float64(2), object(2)
memory usage: 2.5+ KB


### 해당 가게의 모든 리뷰 크롤링

In [116]:
# 가게 이름 누르기
driver.find_element(By.XPATH, f'//*[@id="placeList"]/li[1]/a[1]/span[2]/span[1]/strong').click()

# # 크롤링
# def review_def(url):
#     return driver.find_element(By.XPATH, url).text 

# # 가게 이름
# store = find_text('//*[@id="mArticle"]/div[1]/div/div[1]/h2')
# # 후기 점수
# review_point = find_text('//*[@id="mArticle"]/div[1]/div/div[1]/div[1]/a[1]/span[1]/span[1]')
# # 후기 갯수
# review_count = find_text('//*[@id="mArticle"]/div[1]/div/div[1]/div[1]/a[1]/span[2]/span[1]')
# # 주소
# address = find_text('//*[@id="mArticle"]/div[2]/div/div[1]/div/span[1]')

In [69]:
# 스크롤 아래로 내리기기
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(1)

#다음 더보기 누르기
driver.find_element(By.XPATH, '//*[@id="daumContent"]/div[4]/div[2]/div[3]/a').click()

In [72]:
# 더보기 버튼을 누를 수 있는지 여부 확인하는 함수
def can_click_load_more_button():
    try:
        load_more_button = driver.find_element(By.XPATH, '//*[@id="daumContent"]/div[4]/div[2]/div[3]/a')  # 더보기 버튼의 XPATH
        return load_more_button.is_enabled() and load_more_button.is_displayed()
    except:
        return False

# 더보기 버튼 클릭 함수
def click_load_more_button():
    load_more_button = driver.find_element(By.XPATH, '//*[@id="daumContent"]/div[4]/div[2]/div[3]/a')  # 더보기 버튼의 XPATH
    load_more_button.click()
    time.sleep(3)  # 버튼을 클릭한 후에 충분한 시간 대기 (페이지가 로딩될 때까지)

# 더보기 버튼을 누를 수 없을 때까지 반복해서 클릭
while can_click_load_more_button():
    click_load_more_button()

In [ ]:
# 페이지에 표시된 정보의 갯수 확인
all_results = driver.find_elements(By.XPATH, '//*[@id="placeList"]/li')
print("페이지에 표시된 정보의 갯수:", len(all_results))

정보의 수
1번 정보
//*[@id="placeList"]/li[1]
2번 정보
//*[@id="placeList"]/li[2]
3번 정보
//*[@id="placeList"]/li[3]

In [68]:
address

['서울 종로구 필운대로5가길 31 (누하동)',
 '서울 종로구 창의문로 149 1층 (부암동)',
 '서울 종로구 필운대로 54 (통인동)',
 '서울 종로구 청계천로 201 1~3층 (예지동)',
 '서울 종로구 북촌로4길 20 연화빌딩 1층 (계동)',
 '서울 종로구 동숭길 36 1층 101호 (동숭동)',
 '서울 종로구 북촌로1길 30-11 지하1층 (안국동)',
 '서울 종로구 율곡로3길 75 1층 (소격동)',
 '서울 종로구 율곡로 51 1층 (안국동)',
 '서울 종로구 수표로28길 21-17 1층 (익선동)',
 '서울 종로구 삼일대로32가길 29-1 (익선동)',
 '서울 종로구 창경궁로 236 이화빌딩 1층 (명륜4가)',
 '서울 종로구 율곡로 45 1층 (안국동)',
 '서울 종로구 돈화문로11가길 59 현대뜨레비앙 1층 111호 (익선동)',
 '서울 종로구 성균관로1길 6-6 (명륜3가)']

In [50]:
# 크롤링
def find_text(url):
    return driver.find_element(By.XPATH, url).text 

# 가게 이름
store = find_text('//*[@id="mArticle"]/div[1]/div/div[1]/h2')
# 후기 점수
review_point = find_text('//*[@id="mArticle"]/div[1]/div/div[1]/div[1]/a[1]/span[1]/span[1]')
# 후기 갯수
review_count = find_text('//*[@id="mArticle"]/div[1]/div/div[1]/div[1]/a[1]/span[2]/span[1]')
# 주소
address = find_text('//*[@id="mArticle"]/div[2]/div/div[1]/div/span[1]')